In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from PIL import Image
from tabulate import tabulate
import numpy as np
import json
from IPython.display import display, HTML

In [ ]:
"""
Limitations:
-Time is not a part of any calculations, eg. Crown of the Shattered Queen does not provide damage resistance. This also means that DPS is not calculated - only damage is.
-The damage reduction from Plated Steelcaps, Warden's Mail etc. are currently not supported, but can easily be implemented.
"""

In [2]:
wits_end_on_hit = lambda champion, target, i : (15 + 80/17 * (champion.lvl - 1), "magic")
kraken_slayer_on_hit = lambda champion, taret, i: (60 + champion.stats["bonus attack damage"] * 0.45, "true") if i % 3 == 0 else (0, "phys")
rage_blade_on_hit = lambda champion, target, i: (min(champion.item_stats["critical strike"], 1)*100 * 2, "phys")
borg_on_hit = lambda champion, target, i: (max(target.stats["health"] * 0.08, 15), "phys")
on_hit_map = {"Wit's End": wits_end_on_hit, "Kraken Slayer": kraken_slayer_on_hit, "Guinsoo's Rageblade": rage_blade_on_hit, "Blade of the Ruined King": borg_on_hit}

class Item:
	def __init__(self, name, rarity, stats, cost, mythic_bonus = None):
		self.name = name
		self.cost = cost
		self.stats = stats
		self.rarity = rarity
		self.mythic_bonus = mythic_bonus

In [3]:
accepted_stats = ['attack range', 'mana', 'lethality', 'tenacity', 'heal and shield power', 'mana regeneration', 
                                'attack speed', 'life steal', 'critical strike', 'critical damage','movement speed', 'movement speed percent', 'ability power', 
                                'armor penetration', 'magic penetration', 'magic penetration percent', 'attack damage', 'omnivamp', 'armor', 
                                'ability haste', 'health regeneration', 'slow resist', 'health', 'magic resistance']

def read_item_data(link, rarity = "non-mythic"):
  item_page = requests.get(link)
  item_soup = BeautifulSoup(item_page.content, 'html.parser')

  gold_sections = item_soup.find_all(class_="pi-horizontal-group-item pi-data-value pi-font pi-border-color pi-item-spacing")
  for section in gold_sections:
    if section.get("data-source") == "buy":
      cost = int(section.text)
      break

  stat_header = item_soup.find(class_= "pi-item pi-header pi-secondary-font pi-item-spacing pi-secondary-background")
  stat_rows = stat_header.find_next_siblings(class_= "pi-item pi-data pi-item-spacing pi-border-color") 
  
  item_stats = {}
  for x in stat_rows:
    stat_name = x.find("a")["title"].lower()
    stat_value = x.text[2:x.text.index(" ")]
    try:
      if "%" in stat_value:
        stat_value = stat_value.replace("%", "")
        stat_value = int(stat_value)/100 
        #Handling Void Staff
        if stat_name == "magic penetration":
          stat_name = "magic penetration percent"  
        if stat_name == "movement speed":
          stat_name = "movement speed percent"     
      else:
          stat_value = int(stat_value)
    except Exception as e:
        print("Cannot handle", x.text)
    
    if stat_name in accepted_stats:
      item_stats[stat_name] = stat_value
    else:
      print(stat_name, "is not accepted")
  if rarity != "mythic":
    return item_stats, cost

  mythic_bonus = {}
  if rarity == "mythic":
    passive = item_soup.find(class_="template_sbc", text = "Mythic Passive:")
    passive_stats = [s.text for s in passive.find_next_siblings("span")]
    for stat in passive_stats:
      end_value = stat.index(" ")
      stat_name = stat[end_value:].replace("bonus", "").strip()
      stat_value = stat[:end_value]
      try:
        if "%" in stat_value:
          stat_value = stat_value.replace("%", "")
          stat_value = int(stat_value)/100
          if stat_name == "movement speed":
            stat_name = "movement speed percent" 
        else:
            stat_value = int(stat_value)
      except Exception as e:
        print("Cannot handle", stat, e)
      
      if stat_name in accepted_stats:
        mythic_bonus[stat_name] = stat_value
      else:
        print(stat_name, "is not accepted")
      
    
  return item_stats, cost, mythic_bonus

In [4]:
def construct_items_dict(update_json):
  items_path = "items.json"


  basic_list = []
  legendary_list = []
  mythic_list = []
  all_items = {}
  if update_json:
    #Constructing dictionary containing pairs {item name : item object}
    items_page = requests.get("https://leagueoflegends.fandom.com/wiki/Item_(League_of_Legends)")
    items_soup = BeautifulSoup(items_page.content, 'html.parser')
    item_grid = items_soup.find(id="item-grid")
    sections = [sec.text for sec in item_grid.find_all("dl")]

    item_lists = item_grid.find_all(class_="tlist")
    for i, l in enumerate(item_lists):
      x = l.find_all(class_ ="item-icon")
      for e in x:
        name = e.get("data-item")
        print(name)
        link = "https://leagueoflegends.fandom.com/" + e.find("a")["href"]
        if sections[i] in ["Starter items", "Boots", "Basic items", "Epic items"]:
          stats, cost = read_item_data(link)
          item = Item(name, "basic", stats, cost)
          basic_list.append(item)
          all_items[name] = item
        elif sections[i] == "Legendary items":
          stats, cost = read_item_data(link)
          item = Item(name, "legendary", stats, cost)
          legendary_list.append(item)
          all_items[name] = item
        elif sections[i] == "Mythic items":
          stats, cost, mythic_bonus = read_item_data(link, rarity="mythic")
          item = Item(name, "mythic", stats, cost, mythic_bonus=mythic_bonus)
          mythic_list.append(item)
          all_items[name] = item
    with open(items_path, 'w+') as fp:
      serializable = {item: all_items[item].__dict__ for item in all_items}
      json.dump(serializable, fp)
  else:
    with open(items_path, 'r') as fp:
      items_json = json.load(fp)
      all_items = {}
      for item in items_json:
        i = items_json[item]
        all_items[item] = Item(i["name"], i["rarity"], i["stats"], i["cost"])
        if i["mythic_bonus"] != None:
          all_items[item].mythic_bonus = i["mythic_bonus"]
      for item in all_items:
        if all_items[item].rarity == "basic":
          basic_list.append(item)
        elif all_items[item].rarity == "legendary":
          legendary_list.append(item)
        elif all_items[item].rarity == "mythic":
          mythic_list.append(item)
  return all_items, basic_list, legendary_list, mythic_list

In [5]:
all_items, basic_list, legendary_list, mythic_list = construct_items_dict(False)

In [6]:
class Champion:
  def __init__(self, name, lvl, ability_map, rune_shards, items):
    self.lvl = lvl
    self.name = name
    self.ability_map = ability_map
    self.rune_shards = rune_shards
    self.items = items
    self.item_names = [item.name for item in items]

    if ability_map:
      self.abilities = dict(zip(["q", "w", "e", "r"], self.get_ability_lvls()))

    self.base_stats = self.get_base_stats()
    self.item_stats = self.get_item_stats()
    self.rune_stats = self.get_rune_stats()

    #Combining all sources of stats
    self.stats = dict.fromkeys(accepted_stats, 0)
    for stat in self.base_stats:
      self.stats[stat] += self.base_stats[stat]
    for stat in self.item_stats:
      self.stats[stat] += self.item_stats[stat]
    
    for stat in self.rune_stats:
      self.stats[stat] += self.rune_stats[stat]
    

    #Additional stats
    self.stats["critical damage"] = 1.75
    if self.has_item("Infinity Edge") and self.stats["critical strike"] >= 0.6:
      self.stats["critical damage"] += 0.35
    if self.has_item("Guinsoo's Rageblade"):
      self.stats["critical strike"] = 0
    self.stats["maximum health"] = self.stats["health"]
    self.stats["bonus attack damage"] = self.get_stat(self.item_stats, "attack damage") + self.get_stat(self.rune_stats, "attack damage")
    self.stats["bonus health"] = self.get_stat(self.item_stats, "health") + self.get_stat(self.rune_stats, "health")
    
  def get_stat(self, stat_source, stat):
    if stat_source == {} or stat not in stat_source:
      return 0
    return stat_source[stat]

  def get_ability_lvls(self):
      q = w = e = r = 0
      for lvl in range(1, self.lvl + 1):
        if self.ability_map[lvl] == "q":
          q += 1
        elif self.ability_map[lvl] == "w":
          w += 1	
        elif self.ability_map[lvl] == "e":
          e += 1
        elif self.ability_map[lvl] == "r":
          r += 1
      return (q, w, e, r)

  def set_items(self, items):
    self.items = items

  def set_target(self, target):
    self.target = target

  def get_adaptive(self):
    item_ad = self.get_stat(self.item_stats, "attack damage")
    item_ap = self.get_stat(self.item_stats, "ability power")
    return "attack damage" if item_ad > item_ap else "ability power" 

  def get_rune_stats(self):
    rune_stats = {}
    adaptive = self.get_adaptive()
    power_map = {"force": 5.4 if adaptive == "attack power" else 9, 
                "attack speed": 0.1, "ability haste": 8, "armor": 6, "magic resistance": 8, "health": 15 + (self.lvl - 1) * 125/17}
    for rune in self.rune_shards:
      r = rune
      if rune == "force":
        r = adaptive
      if r in rune_stats:
        rune_stats[r] += power_map[rune]
      else:
        rune_stats[r] = power_map[rune]
    return rune_stats

  def get_item_stats(self):
    item_stats = {}
    legendary_count = 0
    mythic = None
    for item in self.items:
      legendary_count += (item.rarity == "legendary")

      if item.rarity == "mythic": mythic = item

      for stat in item.stats:
        if stat in item_stats:
          item_stats[stat] += item.stats[stat]
        else:
          item_stats[stat] = item.stats[stat]

    if mythic:
      for bonus in mythic.mythic_bonus:
        if bonus in item_stats:
          item_stats[bonus] += legendary_count * mythic.mythic_bonus[bonus]
        else:
          item_stats[bonus] = legendary_count * mythic.mythic_bonus[bonus]
    return item_stats

  def get_base_stats(self):
    search_name = self.name.replace(" ", "")
    c = f"http://ddragon.leagueoflegends.com/cdn/12.2.1/data/en_US/champion/{search_name}.json"
    r = requests.get(c, verify = False)
    d = r.json()
    stats = d["data"][search_name]["stats"]
    hp = stats["hp"] + (self.lvl - 1) * stats["hpperlevel"]
    hp_regen = stats["hpregen"] + (self.lvl - 1) * stats["hpregenperlevel"]
    mana = stats["mp"] + (self.lvl - 1) * stats["mpperlevel"]
    mana_regen = stats["mpregen"] + (self.lvl - 1) * stats["mpregenperlevel"]
    ms = stats["movespeed"]
    armor = stats["armor"] + (self.lvl - 1) * stats["armorperlevel"]
    mr = stats["spellblock"] + (self.lvl - 1) * stats["spellblockperlevel"]
    range = stats["attackrange"]
    attack_damage = stats["attackdamage"] + (self.lvl - 1) * stats["attackdamageperlevel"]
    attack_speed = (self.lvl - 1) * stats["attackspeedperlevel"] / 100

    return {"health": hp, "health regeneration": hp_regen, "mana": mana, "mana regeneration": mana_regen, "movement speed": ms, "armor": armor, "magic resistance": mr, "attack range": range, "attack damage": attack_damage, "attack speed": attack_speed}

  def print_info(self):
    print(f"Level {self.lvl}")
    print(f"Q: {self.abilities['Q']['abilityLevel']}. E: {self.abilities['E']['abilityLevel']}")
    print(f"Items: ", ", ".join(self.item_names))

  def has_item(self, item_name):
    return item_name in self.item_names
  
  def total_item_cost(self):
    return sum([item.cost for item in self.items])

  def use_press_the_attack(self):
    dmg = 40 + 140/17 * (self.lvl - 1)
    self.target.take_dmg(dmg, "phys", "Press the Attack", False, self)
    self.target.exposed = True
    self.target.expose_multiplier = 1.08 + 0.04/17 * (self.lvl - 1)

  def use_galeforce(self):
    dmg = 180 if self.lvl < 10 else 180 + 15 * (self.lvl - 9)
    missing_health_multiplier = min(1 + 5/7*(1 - self.target.stats["health"]/self.target.stats["maximum health"]), 1.7)
    dmg *= missing_health_multiplier
    self.target.take_dmg(dmg, "magic", "Galeforce", False, self)

  def use_shield_bash(self):
    shield = 40 + 260/17*(self.lvl - 1) + self.stats["bonus attack damage"] * 0.4
    dmg = 5 + 25/17 * (self.lvl - 1) + shield * 0.085 + self.stats["bonus health"] * 0.015
    dmg_type = "phys" if self.get_adaptive() == "attack damage" else "magic"
    self.target.take_dmg(dmg, dmg_type, "Shield Bash", False, self)

  def use_ignite(self):
    self.target.take_dmg(50 + 20 * self.lvl, "true", "Ignite", False, self)

  def proc_borg_effect(self):
    self.target.take_dmg(40 + 110/17 * (self.lvl - 1), "magic", "Blade of the Ruined King (cd proc)", False, self)

  def proc_eclipse(self):
    self.target.take_dmg(self.target.max_hp * 0.06, "phys", "Eclipse (cd proc)", False, self)

  def proc_sunderer(self):
    self.target.take_dmg(self.target.max_hp * 0.18, "phys", "Double proc Sunderer", False, self)

  def proc_energized(self):
    self.target.take_dmg(120, "magic", "Energized", False, self)

  def apply_on_hit(self, coefficient):
    for item in self.item_names:
      if item in on_hit_map:
        (dmg, dmg_type), source = on_hit_map[item](self, self.target, self.target.hit_count), item
        if dmg != 0:
          l = 1
          if self.has_item("Guinsoo's Rageblade") and self.target.hit_count % 3 == 0:
            if item != "Kraken Slayer":
              l = 2
          for i in range(l):
            self.target.take_dmg(dmg*coefficient, dmg_type, source, False, self)

    if self.target.hit_count == 2:
      if self.has_item("Eclipse"):
        self.proc_eclipse()
      if self.has_item("Sunderer"):
        self.proc_sunderer()

    if self.target.hit_count == 3:
      self.use_dirty_fighting()
      if self.has_item("Blade of the Ruined King"):
        self.proc_borg_effect()
      self.use_press_the_attack()

    if self.target.hit_count == 4:
      self.use_shield_bash()
  

In [7]:
class Akshan(Champion):
	def __init__(self, lvl, ability_map, rune_shards, items):
		super().__init__("Akshan", lvl, ability_map, rune_shards, items)

	def reset(self):
		self.__init__(self.lvl, self.ability_map, self.rune_shards, self.items)

	def use_auto_1(self):
		dmg = self.stats["attack damage"] * (1 + (self.stats["critical strike"] * (0.75 + self.has_item("Infinity Edge") * 0.35 * (self.stats["critical strike"] >= 0.6))))
		self.target.take_dmg(dmg, "phys", "Auto 1", True, self)
		self.apply_on_hit(1)					

	def use_auto_2(self):
		self.target.take_dmg(self.stats["attack damage"] * 0.5 * (1 + (self.stats["critical strike"] * 0.6125 + (self.has_item("Infinity Edge") * 0.1225 * (self.stats["critical strike"] >= 0.6)))), "phys", "Auto 2", True, self)
		self.apply_on_hit(1)

	def use_q(self, both = True):
		dmg = 5 + (self.abilities["q"] - 1) * 20 + self.stats["attack damage"] * 0.8
		if both: dmg *= 2
		self.target.take_dmg(dmg, "phys", "Avengerang", False, self)

	def use_e(self, bullets):
		base_dmg = 30 + (self.abilities["e"] - 1) * 15
		bonus_ad_dmg = self.stats["bonus attack damage"] * 0.175
		attack_speed_dmg_multiplier = (1 + self.stats["attack speed"] * 0.3)
		crit_multiplier = (1 + (self.stats["critical strike"] * 0.575 + (self.has_item("Infinity Edge") * 0.315 * (self.stats["critical strike"] >= 0.6))))
		bullet_dmg = (base_dmg + bonus_ad_dmg) * attack_speed_dmg_multiplier * crit_multiplier

		for i in range(bullets):
			self.target.take_dmg(bullet_dmg, "phys", "Heroic Swing", True, self)
			self.apply_on_hit(0.25)

	def use_r(self, bullets):
		b = (20 + (self.abilities["r"] - 1)*5)
		crit_multiplier = (1 + (0.5 + 0.175 * self.has_item("Infinity Edge"))*self.stats["critical strike"])
		for bul in range(bullets):
			missing_health_multiplier = 1 + 0.03*(1 - self.target.stats["health"]/self.target.stats["maximum health"])
			dmg = (b + 0.1*self.stats["attack damage"]) * crit_multiplier * missing_health_multiplier
			self.target.take_dmg(dmg, "phys", "Comeuppance", False, self)
		
	def use_dirty_fighting(self):
		dmg = 10 + (min(self.lvl, 8) - 1)*5
		if self.lvl > 8:
			dmg += (min(self.lvl, 14) - 8) * 10
		if self.lvl > 14:
			dmg += (self.lvl - 14) * 15
		self.target.take_dmg(dmg, "magic", "Dirty Fighting", False, self)

	def use_combo(self, combo, p = True):
		if p:
			print(combo)

		for action in combo:
			a = action[0]
			s = 0 if len(action) == 1 else int(action[1:])

			if a == "A":
				self.use_auto_1()
				if s != 1:
					self.use_auto_2()

			elif a == "Q":
				self.use_q(both = s != 1)

			elif a == "E":
				self.use_e(bullets = s)

			elif a == "R":
				self.use_r(bullets = s)

			elif a == "I":
				self.use_ignite()

			elif a == "G" and self.has_item("Galeforce"):
				self.use_galeforce()

			elif a == "S":
				if self.has_item("Rapid Firecannon"):
					self.proc_energized()
				if self.has_item("Stormrazor"):
					self.proc_energized()

In [8]:
class Log:
  def __init__(self):
    self.raw = []
    self.total_damage = 0
  
  def register(self, dmg, dmg_type, source):
    self.raw.append((dmg, dmg_type, source))
    self.total_damage += dmg

  def get_damage_distribution(self):
    distribution = dict.fromkeys(["phys", "magic", "true"], 0)
    for dmg, dmg_type, _ in self.raw:
      distribution[dmg_type] += dmg
    return distribution
  
  def get_source_distribution(self):
    source_distribution = {}
    for dmg, _, source in self.raw:
      if source in source_distribution:
        source_distribution[source] += dmg
      else:
        source_distribution[source] = dmg
    return source_distribution


In [9]:
class Target(Champion):
	def __init__(self, name, lvl, rune_shards, items):
		super().__init__(name, lvl, None, rune_shards, items = items)
		self.log = Log()
		self.dirty_fightning = 0 
		self.hit_count = 0
		self.magic_dmg_taken = 0
		self.physical_dmg_taken = 0
		self.true_dmg_taken = 0
		self.coup_de_grace_multiplier = 1
		self.expose_multiplier = 1
		self.black_cleaver_stacks = 0

	def reset(self):
		self.__init__(self.name, self.lvl, self.rune_shards, self.items)

	def take_dmg(self, dmg, dmg_type, source, hit, champion):
		armor_pen_percent = champion.stats["armor penetration"]
		armor_pen_flat = champion.stats["lethality"]
		black_cleaver_pen = self.black_cleaver_stacks * 0.05
		magic_pen_percent = champion.stats["magic penetration percent"]
		magic_pen_flat = champion.stats["magic penetration"]
		d = dmg#Default true damage
		if dmg_type == "phys":
			ldr_multiplier = max(1, 1 + min(0.000075 * (self.stats["maximum health"] - champion.stats["maximum health"]), 0.15) * champion.has_item("Lord Dominik's Regards"))
			d =  100/(100 + max((self.stats["armor"] * (1 - armor_pen_percent) * (1 - black_cleaver_pen) - armor_pen_flat), 0)) * dmg * self.expose_multiplier * self.coup_de_grace_multiplier * ldr_multiplier
			if self.black_cleaver_stacks < 6 and champion.has_item("Black Cleaver"):
				self.black_cleaver_stacks += 1
		elif dmg_type == "magic":
			liandrys_multiplier = max(1, 1 + min(1.2/12500 * (self.stats["maximum health"] - champion.stats["maximum health"]), 0.12)) * champion.has_item("Liandry's Anguish")
			d =  100/(100 + max((self.stats["magic resistance"] * (1 - magic_pen_percent) - magic_pen_flat), 0)) * dmg * self.expose_multiplier * self.coup_de_grace_multiplier
			
		self.stats["health"] = max(self.stats["health"] - d, 0)
		self.log.register(d, dmg_type, source)

		if self.stats["health"] < self.stats["maximum health"] * 0.05 and self.stats["health"] > 0:
			self.take_dmg(self.stats["health"], "true", "The Collector", False, champion)

		if hit:
			self.hit_count += 1
		if self.stats["health"] < self.stats["maximum health"] * 0.4:
			self.coup_de_grace_multiplier = 1.08

In [20]:
def get_item_dataframe(champion, targets, combos):
  columns = []
  for target in targets:
    col_name = f'{target.name}: a={target.stats["armor"]}, m=f{target.stats["magic resistance"]}, hp=f{target.stats["maximum health"]}'
    columns.append(col_name)
  
  index = []
  for i, combo in enumerate(combos):
    index.append(f"combo {i} dmg")
    index.append(f"combo {i} dmg/g")

  data = []
  for target in targets:
    col_vals = []
    for combo in combos:
      champion.set_target(target)
      champion.use_combo(combo, p = False)

      col_vals.append(target.log.total_damage)
      dmg_per_gold = target.log.total_damage / champion.total_item_cost()
      col_vals.append(dmg_per_gold)
      target.reset()
    data.append(col_vals)
  
  df = pd.DataFrame(np.array(data).T, index = index, columns = columns)
  return df

def item_analysis(num_items, champion, targets, combos, item_subset_):
  current_items = champion.items
  illegal_pairs = [("Infinity Edge", "Guinsoo's Rageblade"), ("Lord Dominik's Regards", "Serylda's Grudge")]
  #Automatically filtering item_subset to reduce size
  #Removes mythic if the champion already has a mythic
  #Removes legendaries that the champion already has 
  has_mythic = False
  for item in current_items:
    if item.rarity == "mythic":
      has_mythic = True
  item_subset = []
  for item in item_subset_:
    if has_mythic and item.rarity == "mythic":
      continue
    if item in current_items and not item.rarity == "basic":
      continue
    item_subset.append(item)

  item_combinations = {}
  def rec(depth, item_subset, item_comb, current_idx):
    if len(item_comb) == num_items:
      item_names = [item.name for item in item_comb]
      illegal_pair = False
      for i1, i2 in illegal_pairs:
        if i1 in item_names and i2 in item_names:
          illegal_pair = True
      
      #Only 1 mythic allowed
      if sum([1 for i in item_comb if i.rarity == "mythic"]) > 1:
        illegal_pair = True

      if not illegal_pair:
        champion.set_items(item_comb)
        champion.reset()
        df = get_item_dataframe(champion, targets, combos)
        items_as_key = ", ".join(sorted(item_names))
        item_combinations[items_as_key] = df.copy()

    if depth > 0:
      for i in range(current_idx, len(item_subset)):
        rec(depth - 1, item_subset, item_comb + [item_subset[i]], i+1)
  for target in targets:
    target.reset()

  rec(num_items, item_subset, current_items, 0)
  return item_combinations

 

In [21]:
def filter_item_subset(item_subset, stats):
  filtered = []
  for item in item_subset:
    for stat in stats:
      if stat in item.stats:
        filtered.append(item)
        break
  return filtered

def get_items(item_names):
  return [all_items[item_name] for item_name in item_names]

def item_comb_query(analysis, item_names):
  items_as_key = ", ".join(sorted(item_names))
  return analysis[items_as_key] 

def get_best_comb(analysis, criteria):
  best_comb = None
  best_score = 0
  for item_comb in analysis:
    df = analysis[item_comb]
    score = df.loc[criteria][2]
    if score > best_score:
      best_score = score
      best_comb = item_comb
  return best_comb

In [22]:
def quick_comparison(num_items, champion, targets, combos, comparison_items):
  analysis = item_analysis(num_items, champion, targets, combos, comparison_items)
  dfs = []
  for comb in analysis:
    df = analysis[comb]
    df.loc["item comb"] = comb 
    except_last = df.iloc[:-1]
    last = df.iloc[-1:]
    new_total = pd.concat([last, except_last])
    dfs.append(new_total)
  

  display(HTML(pd.concat(dfs).to_html()))

In [23]:
combos = [
	["A", "S", "E4", "Q", "G", "A", "R2", "A"],
	["A", "S", "A1", "E4", "Q", "A1", "A", "R4"],
	["A", "E4", "S", "Q", "A", "G", "A", "R2", "A"]
]

q_max ={
          1:"q",2:"e",3:"q",4:"w",5:"q",6:"r",
          7:"q",8:"e",9:"q",10:"e",11:"r",
          12:"e",13:"e",14:"w",15:"w",16:"r",
          17:"w", 18:"w"
        }

e_max = {
          1:"q",2:"e",3:"e",4:"w",5:"e",6:"r",
          7:"e",8:"e",9:"q",10:"q",11:"r",
          12:"q",13:"q",14:"w",15:"w",16:"r",
          17:"w", 18:"w"
        }



In [24]:
akshan = Akshan(11, q_max, ["attack speed", "force", "magic resistance"], get_items([]))


targets = [Target("Volibear", akshan.lvl - 2, ["attack speed", "force", "armor"], get_items(["Plated Steelcaps", "Frostfire Gauntlet"])),
           Target("Azir", akshan.lvl +1 , ["attack speed", "force", "armor"], get_items(["Luden's Tempest"]))]
filtered_legendaries = filter_item_subset(get_items(all_items), ["attack damage", "attack speed", "critical strike"])
quick_analysis = quick_comparison(3, akshan, targets, combos, get_items(["Kraken Slayer", "Galeforce", "Immortal Shieldbow", "Wit's End", "Blade of the Ruined King", "Lord Dominik's Regards"]))


,"Volibear: a=114, m=f67.0, hp=f1750","Azir: a=58, m=f35.5, hp=f1564"
item comb,"Blade of the Ruined King, Kraken Slayer, Wit's End","Blade of the Ruined King, Kraken Slayer, Wit's End"
combo 0 dmg,2214.266054,2793.9475
combo 0 dmg/g,0.225946,0.285097
combo 1 dmg,2416.42596,3139.269863
combo 1 dmg/g,0.246574,0.320334
combo 2 dmg,2613.837152,3366.132927
combo 2 dmg/g,0.266718,0.343483
item comb,"Kraken Slayer, Lord Dominik's Regards, Wit's End","Kraken Slayer, Lord Dominik's Regards, Wit's End"
combo 0 dmg,2304.417086,2827.87473
combo 0 dmg/g,0.24257,0.297671
